In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn import svm, linear_model, ensemble
from sklearn import model_selection
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings("ignore")

In [2]:
from scipy.stats import randint, uniform
from pyswarm import pso

In [3]:
labels = pd.read_csv("antibacterial_lable.csv", index_col=0 )
data = pd.read_csv("mibig(antismash).csv", index_col=0 )

In [4]:
def get_sample_responses(vectors, responses):
    samples = vectors
    index = list(set(samples.index).intersection(set(labels.index)))
    responses = responses.loc[index]
    samples = samples.loc[index]
    return samples, responses

In [5]:
X , y = get_sample_responses(data, labels)
bgc_ids = pd.Series(y.index)

In [6]:
def objective(params):

    n_estimators, criterion, min_samples_split, min_samples_leaf, max_features, class_weight = params
    
    criterion = ['gini','entropy'][int(criterion)]
    max_features = ['auto', 'sqrt', 'log2'][int(max_features)]
    class_weight = ['balanced','balanced_subsample'][int(class_weight)]

    clf = ensemble.ExtraTreesClassifier(
        n_estimators=int(n_estimators),
        criterion=criterion,
        min_samples_split=int(min_samples_split),
        min_samples_leaf=int(min_samples_leaf),
        max_features=max_features,
        class_weight=class_weight,
        bootstrap=False,
        n_jobs=-1)
     
    acc_kf = []
    
    cv = KFold(n_splits=5,random_state=0,shuffle=True)
    for i, (id_train, id_val) in enumerate(cv.split(bgc_ids)):
        train_ids, val_ids = bgc_ids[id_train], bgc_ids[id_val]
        X_train, X_val = X.loc[train_ids].values, X.loc[val_ids].values
        y_train, y_val = y.loc[train_ids].values, y.loc[val_ids].values
        #print('Train:', len(X_train), 'Test:', len(X_val))
    
        clf.fit(X_train, y_train)
        pred = clf.predict(X_val)
    
        acc = accuracy_score(pred, y_val)
        acc_kf.append(acc)
        
    acc_kf = np.array(acc_kf).mean()
    
    return -acc_kf

In [7]:
lb = [1, 0, 2, 1, 0, 0] 
ub = [200, 1, 50, 10, 2, 1] 

In [8]:
%%time
best_params, best_score = pso(objective, lb, ub, swarmsize=50, maxiter=1)

Stopping search: maximum iterations reached --> 1
CPU times: user 4min 45s, sys: 17.9 s, total: 5min 3s
Wall time: 1min 44s
